# Data Challenges in Calculating Historical Stock Returns

In [ ]:
# hide
import logging

logging.getLogger("yfinance").setLevel(logging.CRITICAL)

In [ ]:
import pandas as pd
from skfin.plot import *
from tqdm.auto import tqdm

In this section, we discuss two issues related to stock data:
- survivorship bias
- matching datasets (e.g. tickers and company names). 

Survivorship bias in data analysis refers to the error that occurs when only the "surviving" or currently existing subjects are considered, while those that have ceased to exist or dropped out are ignored. This leads to overly optimistic results and conclusions because the analysis does not account for entities that failed, exited, or disappeared from the sample. For example, in financial studies, analyzing only funds that exist today ignores those that closed down due to poor performance, thereby distorting performance statistics.

Matching stock tickers and company names presents several challenges. First, ticker symbols can change over time due to corporate actions such as mergers, acquisitions, or rebranding, making it difficult to maintain a consistent mapping. Additionally, different stock exchanges may assign the same or similar tickers to entirely different companies, leading to ambiguity, especially in multinational datasets. Company names themselves can vary due to abbreviations, punctuation, company suffixes (like Inc., Corp., Ltd.), or changes following corporate events. Name spelling differences, typographical errors, and the use of aliases or former names further complicate matching. 

## Returns data 

### Scikit-learn returns

The main issue with this dataset compiled by `Scikit-learn` is we have no information on how the returns were constructed (and how the stocks were chosen at the time). 

In [ ]:
from skfin.dataloaders.constants.mappings import symbol_dict
from skfin.datasets_ import load_sklearn_stock_returns

clip = lambda x: x.T.clip(
    lower=x.quantile(q=0.01, axis=0), upper=x.quantile(0.99, axis=0), axis=0
).T
rescale = lambda x: x.div(x.std())

In [ ]:
ret = load_sklearn_stock_returns(cache_dir="data") / 100
ret_clip = ret.pipe(clip)

### Yahoo finance returns

The main issue with Yahoo finnace returns is the survivorship bias because the deslisted firms are no longer available in the past.

In [ ]:
import yfinance as yf

In [ ]:
tickers = list(symbol_dict.keys())
start, end = ret.index[0].strftime("%Y-%m-%d"), ret.index[-1].strftime("%Y-%m-%d")

In [ ]:
data = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)
data = data.dropna(how="all", axis=1)

In [ ]:
tickers_ = data.columns.get_level_values(1).unique()
dividends = {}
for ticker in tickers_:
    stock = yf.Ticker(ticker)
    div = stock.dividends[start:end]
    if div.dtype == "O":
        div = div.str.replace(" USD", "").astype(float)
    dividends[ticker] = div

div = (
    pd.concat(dividends, axis=1)
    .pipe(lambda x: x.set_index(x.index.tz_localize(None)))
    .reindex_like(data["Close"])
    .fillna(0)
    .astype(float)
    .div(data["Close"])
)

## Survivorship bias and non-US firms

In [ ]:
cols = ["CAJ", "SNE", "UN", "NAV", "TOT", "RTN", "DELL", "YHOO", "CVC"]
{c: symbol_dict[c] for c in cols}

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
line(ret[cols], cumsum=True, ax=ax[0], title="Unclipped returns", loc="best")
line(ret_clip[cols], cumsum=True, ax=ax[1], title="Clipped returns", loc="best")

### Dirty data and jumps 

Total returns are defined as: 

$$ total\_returns_t = \frac{price_t + dividend_t - price_{t-1}}{price_{t-1}}.$$ 

Adjustments are necessary when there are stock splits (or reverse splits) that alter the number of shares outstanding and the nominal price per share.

In [ ]:
data.columns.get_level_values(0).unique()

In [ ]:
pret = data["Close"].pct_change().dropna(how="all", axis=1)
tret = pret.add(div)

In [ ]:
rets = {
    "price returns": pret,
    "total returns": tret,
}

In [ ]:
axis = 0
pd.DataFrame({k: v.corrwith(ret, axis=axis) for k, v in rets.items()}).mean()

In [ ]:
axis = 0
pd.DataFrame({k: v.corrwith(ret_clip, axis=axis) for k, v in rets.items()}).mean()

In [ ]:
cols = ["IBM", "AAPL", "BAC", "JPM", "WFC", "PG"]
fig, axs = plt.subplots(2, 3, figsize=(20, 8))
axs = axs.ravel()
for i, c in enumerate(cols):
    line(
        pd.concat(
            {
                "pret (yfinance)": pret[c],
                "tret (yfinance)": tret[c],
                "sklearn": ret[c],
                "sklearn (clipped)": ret_clip[c],
            },
            axis=1,
        )
        .fillna(0)
        .pipe(rescale),
        cumsum=True,
        title=c,
        ax=axs[i],
        loc="best",
    )

## Matching to other datasets 

Matching company names is often a time-consuming task: below we use the package `rapidfuzz` to check the candidate matches. 

In [ ]:
from skfin.datasets_ import load_10X_summaries
from skfin.dataloaders.constants.mappings import mapping_10X

df = load_10X_summaries()

In [ ]:
v = mapping_10X["CVC"]
print(v)
df.loc[lambda x: x.CoName.isin(v if isinstance(v, list) else [v])].loc[
    ret.index[0] : ret.index[-1]
].loc["2006"].iloc[:, :10]

In [ ]:
CoName = list(
    df.assign(CoName=lambda x: x.CoName.str.upper())
    .groupby(["date", "CoName"])["FILING_DATE"]
    .count()
    .loc[ret.index[0] : ret.index[-1]]
    .groupby(level=1)
    .count()
    .index
)

from rapidfuzz import fuzz

pd.Series({c: fuzz.token_set_ratio("CABLEVISION", c) for c in CoName}).sort_values(
    ascending=False
).head(5)